**Stack**
stacking is referred as transforming horizontal column names to vertical column values . stack function converts the data from a wide to long format
level: Prescribed level(s) to stack from column axis onto index axis (int,str, list, default -1).
dropna: Whether to drop rows with missing values in the resulting frame, defaults (bool, default True).

**Unstack**
Unstacking converts a data frame from long to wide format. By default it takes the inner most index values and returns a data frame by reshaping them as the columns.
level: level(s) of index to unstack, defaults to -1 (int or string or list of these)
fill_value: replace NAN with value specified if unstack produces missing values.

In [1]:
import pandas as pd

In [2]:
data = pd.DataFrame({
    'Sensor':['sensor1','sensor1','sensor1','sensor2','sensor2','sensor2'],
    'Metric':['Pressure','Tempreature','Flow','Pressure','Tempreature','Flow'],
    '2017':[79,40,20,59,29,54],
    '2018':[43,56,23,65,76,34]
})

data

,Sensor,Metric,2017,2018
0,sensor1,Pressure,79,43
1,sensor1,Tempreature,40,56
2,sensor1,Flow,20,23
3,sensor2,Pressure,59,65
4,sensor2,Tempreature,29,76
5,sensor2,Flow,54,34


In [5]:
data.set_index(['Sensor', 'Metric']).stack().reset_index().rename(columns={'level_2': 'Year', 0: 'Value'})

,Sensor,Metric,Year,Value
0,sensor1,Pressure,2017,79
1,sensor1,Pressure,2018,43
2,sensor1,Tempreature,2017,40
3,sensor1,Tempreature,2018,56
4,sensor1,Flow,2017,20
5,sensor1,Flow,2018,23
6,sensor2,Pressure,2017,59
7,sensor2,Pressure,2018,65
8,sensor2,Tempreature,2017,29
9,sensor2,Tempreature,2018,76


In [6]:
data.stack()

0  Sensor        sensor1
   Metric       Pressure
   2017               79
   2018               43
1  Sensor        sensor1
   Metric    Tempreature
   2017               40
   2018               56
2  Sensor        sensor1
   Metric           Flow
   2017               20
   2018               23
3  Sensor        sensor2
   Metric       Pressure
   2017               59
   2018               65
4  Sensor        sensor2
   Metric    Tempreature
   2017               29
   2018               76
5  Sensor        sensor2
   Metric           Flow
   2017               54
   2018               34
dtype: object

In [15]:
data.set_index(['Metric']).stack()
#data.set_index(['Sensor', 'Metric'])

Metric             
Pressure     Sensor    sensor1
             2017           79
             2018           43
Tempreature  Sensor    sensor1
             2017           40
             2018           56
Flow         Sensor    sensor1
             2017           20
             2018           23
Pressure     Sensor    sensor2
             2017           59
             2018           65
Tempreature  Sensor    sensor2
             2017           29
             2018           76
Flow         Sensor    sensor2
             2017           54
             2018           34
dtype: object

In [17]:
df_sensors = data.set_index(['Sensor', 'Metric'])
df_sensors.columns = pd.MultiIndex.from_product([['Value'], df_sensors.columns])
df_sensors

Value     
                     2017 2018
Sensor  Metric                
sensor1 Pressure       79   43
        Tempreature    40   56
        Flow           20   23
sensor2 Pressure       59   65
        Tempreature    29   76
        Flow           54   34

In [18]:
df_sensors.stack(level=1)

Value
Sensor  Metric                 
sensor1 Pressure    2017     79
                    2018     43
        Tempreature 2017     40
                    2018     56
        Flow        2017     20
                    2018     23
sensor2 Pressure    2017     59
                    2018     65
        Tempreature 2017     29
                    2018     76
        Flow        2017     54
                    2018     34

**Unstack function**

In [19]:
df_stacked = (data.set_index(['Sensor', 'Metric'])
              .stack()
              .reset_index()
              .rename(columns={'level_2': 'Year', 0: 'Value'}))
df_stacked

,Sensor,Metric,Year,Value
0,sensor1,Pressure,2017,79
1,sensor1,Pressure,2018,43
2,sensor1,Tempreature,2017,40
3,sensor1,Tempreature,2018,56
4,sensor1,Flow,2017,20
5,sensor1,Flow,2018,23
6,sensor2,Pressure,2017,59
7,sensor2,Pressure,2018,65
8,sensor2,Tempreature,2017,29
9,sensor2,Tempreature,2018,76


In [25]:
# Step 1: Set the columns [Sensor, Metric and Year] as index.
# Step 2: Apply unstack.
# Step 3: Process the output to a consumable form.

# Setting the index columns
df_stacked.set_index(['Sensor', 'Metric','Year'])

Value
Sensor  Metric      Year       
sensor1 Pressure    2017     79
                    2018     43
        Tempreature 2017     40
                    2018     56
        Flow        2017     20
                    2018     23
sensor2 Pressure    2017     59
                    2018     65
        Tempreature 2017     29
                    2018     76
        Flow        2017     54
                    2018     34

In [27]:
# Applying unstack
df_stacked.set_index(['Sensor', 'Metric','Year']).unstack()

Value     
Year                 2017 2018
Sensor  Metric                
sensor1 Flow           20   23
        Pressure       79   43
        Tempreature    40   56
sensor2 Flow           54   34
        Pressure       59   65
        Tempreature    29   76

In [29]:
# Processing the output

dfx = (df_stacked
       .set_index(['Sensor', 'Metric', 'Year'])
       .unstack())
dfx.columns = dfx.columns.droplevel([0])
dfx.rename_axis([None], axis=1).reset_index()

,Sensor,Metric,2017,2018
0,sensor1,Flow,20,23
1,sensor1,Pressure,79,43
2,sensor1,Tempreature,40,56
3,sensor2,Flow,54,34
4,sensor2,Pressure,59,65
5,sensor2,Tempreature,29,76


In [35]:
# Unstacking at multiple index levels at a time.

(df_stacked
 .set_index(['Sensor', 'Year', 'Metric'])
 .unstack(level=[-2,-1]))

Value                                           
Year        2017     2018        2017        2018 2017 2018
Metric  Pressure Pressure Tempreature Tempreature Flow Flow
Sensor                                                     
sensor1       79       43          40          56   20   23
sensor2       59       65          29          76   54   34

In [38]:
# Unstacking at multiple index levels at a time.

(df_stacked
 .set_index(['Sensor', 'Year', 'Metric'])
 .unstack(level=[-3,-2]))

Value                  
Sensor      sensor1      sensor2     
Year           2017 2018    2017 2018
Metric                               
Flow             20   23      54   34
Pressure         79   43      59   65
Tempreature      40   56      29   76